In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os.path as path

import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.preprocessing as pre
import librosa
import music21 as mus

import vmo
import vmo.analysis as van
import vmo.generate as vge
# Music analysis
import vmo.utils.chromagram as vchroma
import vmo.utils.music21_interface as vmusic
import vmo.utils.harmonic_changes as vharm
# Logics
import vmo.logics.model_checking as vmodel

In [ ]:
midi_path = path.expanduser('~/Musique/midi/')
midi_file = 'mozart/sonatina_num_4-mvt_1.mid'

"""Oracle generation."""

# Generate music21 stream from MIDI file
stream = mus.converter.parse(midi_path + midi_file)
# Generate VMO from music21 stream
oracle = vmusic.from_stream(stream.flat.notes[:30], framesize=1.0, dfunc='tonnetz')

In [ ]:
vmusic.extract_frame_oracle(stream.flat, oracle, 5).flat.show('text')

In [ ]:
"""Integrated MIDI playback.

Might need some parameters-tweaking to work if midi player binding is not
done straight out-of-the-box with music21.

In that case, run:
>>> from music21 import *
>>> us = environment.UserSettings()
>>> us.create()
>>> us['midiPath'] = <PATH/TO/MIDI/PLAYER>
>>> # Example : us['midiPath'] = u'/usr/bin/totem-audio-preview'
"""

_beginning_notes.show('midi')

In [ ]:
"""Harmonic changes detection and chord progression extraction."""

# Extract the sequences of notes from the stream
notes = stream.flat.notes
# Extract the first 16 quarter-notes
_beginning_notes = vmusic.extract_frame(notes, 0., 16.)
# Compute offsets of harmonic changes (in quarter-length)
harmonic_changes = vharm.from_stream_by_offsets(
    _beginning_notes, framesize=1/4.)
# Compute the chord progression in the extracted frame
prog = vmusic.get_chord_progression(_beginning_notes)

print("First two measures:")
_beginning_notes.show('text')
print("\nExtracted progression:\n" + str(prog))
print("\nHarmonic changes:\n" + str(harmonic_changes))

In [ ]:
import vmo.utils.nuxmv.model as numodel
print(numodel.print_oracle(oracle, original_stream=stream))

In [ ]:
"""Chord-progression extraction."""
# Two sequences of degrees, we'll try to generate a path following the first
# then the second (possibibly with an arbitrary path between both) in `oracle`
simple_progs = [[1, '3', 5, 3, 5], [1, 5, 3, 5, 3]]
gen_path, chosen_tonic = vmodel.make_piecewise_chord_progression_tonic_free(
    oracle, simple_progs, original_stream=stream)

if gen_path:
    print("A path has been geenerated!")
    print("The chosen tonic is: " + chosen_tonic)
    print("\nThe path is the following: \n")
    for state in gen_path:
	print(state)

In [ ]:
# Recompose new stream from extracted path
extracted = vmusic.path_to_stream(notes, gen_path)
# Plot piano roll of the extrated stream
extracted.plot('pianoroll')
